#### 라벨링 하기

In [1]:
import tensorflow
import keras
import pandas as pd
import numpy as np
import requests
import re
from konlpy.tag import *
from collections import Counter

In [4]:
naver_blog_text_scaling_keyword = pd.read_csv(r"C:\vscode\project\잇더컴퍼니\1차프로젝트\naver_blog_text_scaling_keyword.csv")

In [3]:
naver_blog_text_scaling_keyword = pd.read_csv(r"C:\vscode\project\잇더컴퍼니\1차프로젝트\naver_blog_text_scaling_keyword.csv", encoding= "utf-8-sig")

In [6]:
del naver_blog_text_scaling_keyword["Unnamed: 0"]

In [7]:
#키워드가 들어간 키워드 모음보기
def kw_in_blog_k(kw):
    print('"', kw, '"', '포함된 리뷰 총',len(naver_blog_text_scaling_keyword[naver_blog_text_scaling_keyword['주요 키워드'].apply(lambda x:kw in x)]['주요 키워드']),'건')
    return naver_blog_text_scaling_keyword[naver_blog_text_scaling_keyword['주요 키워드'].apply(lambda x:kw in x)]['주요 키워드']

In [9]:
kw_in_blog_k("보신") 

" 보신 " 포함된 리뷰 총 4 건


1      몸보신,요리,똥손,아이,고급,캠핑,여행,영양,선물,가족,보양,간편,명란,버터,계란,...
13                                       가격,고급,간단,몸보신,건강
20                                     가족,아이,저녁,몸보신,집,간단
133                                             몸보신,간편,집
Name: 주요 키워드, dtype: object

In [13]:
#키워드가 들어간 문장보기
def kw_in_blog_t(kw):
    print('"', kw, '"', '포함된 리뷰 총',len(naver_blog_text_scaling_keyword[naver_blog_text_scaling_keyword['변환된 내용'].apply(lambda x:kw in x)]['주요 키워드']),'건')
    return naver_blog_text_scaling_keyword[naver_blog_text_scaling_keyword['변환된 내용'].apply(lambda x:kw in x)]['변환된 내용']

In [14]:
kw_in_blog_t("보신") 

" 보신 " 포함된 리뷰 총 10 건


1      육아하는 RN 널봄맘요리똥손인 제가 환절기를 앞두고 식구들 몸보신을 위해뭘 조리해볼...
3      전복솥밥 키트가 있더라구요. 몸 보신하고 싶어서 최상급 전복 사서 직접 만들어 먹으...
9      국내산 전복솥밥 밀키트 인색전복장 최상급 전복이 가득들어 몸보신에 추천안녕하세요. ...
13     요즘 밀키트가 너무 잘 나오고 있는 거 같아요.간단하게 해먹기 좋으면서 또 신선도도...
20     환절기라 그런지 아이들도 그렇고 온가족이 감기에 걸려서 일주일동안 아팠어요.. ㅠ이...
39     솥밥 밀키트 이야기솥밥밥솥.비가오나눈이오나본가 어미의 품에서떠나 독립을 하게 되면매...
83     6월 블챌 23.06.11  2023.06.286월을 살고 있는 사람 ..바로 나야...
85     문어 효능부터 삶는 방법까지 다 알아봐요  쫄깃하고 맛있는 문어구이 볶음 숙회 튀김...
129    내가 생각했던 5월의 날씨보다훨씬 더습하고 무더위가 시작된한국의 이른 여름.여름.여...
133    짝지가 출장으로 부재중이다보니집에서 집밥다운 집밥을 먹기가 힘들어지는 것 같아요ㅠ_...
Name: 변환된 내용, dtype: object

In [364]:
# 결과 보이는 설정 ....없이 보이게
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None) 

In [353]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

In [16]:
keyword_zip = []
naver_blog_text_scaling_keyword["주요 키워드"].apply(lambda x : keyword_zip.append(x))

0      None
1      None
2      None
3      None
4      None
       ... 
132    None
133    None
134    None
135    None
136    None
Name: 주요 키워드, Length: 137, dtype: object

In [17]:
keyword_zip = ",".join(keyword_zip)

In [18]:
keyword_zip1 = keyword_zip.split(",")

In [19]:
keyword_zip1 = [item for item in keyword_zip1 if item != ''] 

In [20]:
keyword_zip1 = list(set(keyword_zip1))

In [31]:
def person_labeling(x):
    who_guest = ['지인','손님','직장','친구','남친', '남자', '여자'] #1 외부인

    who_mam = ['엄마','어머니', '아빠','아버지','아부지', '할아버지', '할머니', '부모','엄빠'] # 2 어른

    who_kid = ['딸','아들','아이','애들','얘들', '아기','애기'] # 3 아이

    who_husband = ['남편','오빠', '남의편','아저씨', '부부', '와이프', '주부', '신랑'] # 4 부부

    who_fam = ['식구','가족', '동생', '언니'] # 5 가족

    who_solo = ['혼자', '자취', '1인가구', '저는', '저한', '제가', '제 입'] #6 혼자   

    if any(i in x for i in who_guest):
        person_label = 1
    elif any(i in x for i in who_mam):
        person_label = 2
    elif any(i in x for i in who_kid):
        person_label = 3
    elif any(i in x for i in who_husband):
        person_label = 4
    elif any(i in x for i in who_fam):
        person_label = 5
    elif any(i in x for i in who_solo):
        person_label = 6
    else:
        person_label = 6
    
    return person_label

In [32]:
def purpose_labeling(sen):
    invite = ['홈파티','모임','본가','집','야외','소개',"캠핑",'집들이', '초대', '접대', '파티', '해드', '파티', '크리스마스', '대접', '생일', '생신', '손님','모실때', '모시', '어르신', '어루신',"시댁"] # 2
    goodmeal = ['특별식','별미','영양','보양식','집밥','중복','특별한','신혼','기념일','초복','유아식','프리미엄',"기력",'보신', '분위기', '고급', '건강', '보양', '도전', '기운', '기분','제대로','성공',"몸보신","요리"] # 3
    comfortable = ['간단식','자취','육아','이벤트','연휴','혼밥','저렴','급할때','초보','쉬움','왕초보','반찬이 없을때','할인','가성비','반찬','간편','편함','손질','귀찮','편하','급하','간단','밥하기','입맛없','입맛 없','밥맛없','밥맛 없','반찬없','반찬 없','바쁜','바쁠','바쁘',"비쌈","세일"] # 4
    gift = ['선물','드렸','보내','보냈'] # 1 중복 5
    label = 0
    for kw in goodmeal:
        if kw in sen:
            label = 3
            break
    for kw in comfortable:
        if kw in sen:
            if label == 3:
                label = 5
            else:
                label = 4
            break
    for kw in invite:
        if kw in sen:
            label = 2
            break
    for kw in gift:
        if kw in sen:
            label = 1
            break
    return label

In [33]:
def place_labeling(x):
    house = ['집','집밥','자취'] #1

    travel = ['여행'] # 2

    camping = ['캠핑','캠핑장'] # 3

    hospital = ['병원','병문안'] # 4

    if any(i in x for i in hospital):
        place_label = 4
    elif any(i in x for i in camping):
        place_label = 3
    elif any(i in x for i in travel):
        place_label = 2
    elif any(i in x for i in house):
        place_label = 1
    else:
        place_label = 0
    
    return place_label

In [361]:
naver_blog_text_scaling_keyword["person_labeling"] = naver_blog_text_scaling_keyword["주요 키워드"].apply(lambda x : person_labeling(x))

In [ ]:
naver_blog_text_scaling_keyword["purpose_labeling"] = naver_blog_text_scaling_keyword["주요 키워드"].apply(lambda x : purpose_labeling(x))

In [ ]:
naver_blog_text_scaling_keyword["place_labeling"] = naver_blog_text_scaling_keyword["주요 키워드"].apply(lambda x : place_labeling(x))

In [304]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.Collecting openpyxl
     -------------------------------------- 250.0/250.0 kB 2.6 MB/s eta 0:00:00



DEPRECATION: Loading egg at c:\users\nt550\anaconda3\envs\textmining\lib\site-packages\py_hanspell-1.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [305]:
import openpyxl

In [362]:
naver_blog_text_scaling_keyword.to_excel(r"naver_blog_text_scaling_keyword_label_finish2.xlsx")

In [277]:
counts = naver_blog_text_scaling_keyword['person_labeling'].value_counts()

print(counts)

person_labeling
7    51
4    21
1    18
3    15
6    13
2    13
5     6
Name: count, dtype: int64
